In [90]:
from bs4 import BeautifulSoup
import urllib
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.select import Select
import time
import random
import os
import pandas as pd
import csv

## Bills and Budgets

In [105]:
def crawl(value):
    #Web Crawler Setup
    chrome_options = Options()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(executable_path="Chrome/chromedriver.exe", options = chrome_options)

    # The website for the parliamentary info
    parliament_url = 'https://sprs.parl.gov.sg/search/home'

    #Access main Website
    driver.get(parliament_url)

    # Click on the filter tab
    driver.find_element_by_id("advSearchLabel").click()

    # Get the array of web elements with a form control
    forms = driver.find_elements_by_css_selector("select.form-control.ng-untouched")

    # Click on both the 13th and 14th to extract the data
    to_extract = Select(forms[0])

    # Select the dates that are important
    to_extract.select_by_value('13: 13')
    to_extract.select_by_value('14: 14')

    # Select the filter
    to_filter = Select(forms[4])

    # Select the section to be extracted
    to_filter.select_by_value(value)

    # Click on the search button
    driver.find_element_by_xpath("/html/body/app-root/app-search/div[@id='container']/div[@id='divmpscreen2']/div[@class='row']/div[@class='col-sm-12 text-right pull-right']/div[@class='col-sm-12 form-group']/button[@class='btn btn-black'][2]").click()

    # Wait for 3 seconds for the page to load
    driver.implicitly_wait(3)

    # Switch to the other tab with the information
    driver.switch_to.window(driver.window_handles[1])

    # Results dictionary
    results = {}

    # Flag to stop looping
    not_end = True

    count = 0

    # Loop until reach end of page
    while not_end:

        # Wait for 5 seconds
        driver.implicitly_wait(5)

        # Get all the webelements to be clicked later
        output = driver.find_elements_by_css_selector("tbody>tr>td>a")

        for links in output:
            links.click()

            count += 1

            # Switch to the content window
            driver.switch_to.window(driver.window_handles[2])

            # Delete those span with the procText class
            small_words = set([i.text for i in driver.find_elements_by_css_selector('span.procText')])

            # Get the title of the report
            title = driver.find_element_by_css_selector(".titleCaps").text.lower()

            # Get the content of the report
            content = driver.find_elements_by_css_selector('.reportTable > p')

            # Instantiate empty string as temp
            tmp = ''

            # Iterate through the content webelement to get the textual information
            for para in content:

                if para.text not in small_words:
                    tmp += para.text

            # Fill the results
            results[count] = {'title': title, 'text': tmp}

            # Close current page
            driver.close()

            # Go back to the main results page
            driver.switch_to.window(driver.window_handles[1])

        # Iterate through the pages
        try:
            # Try to go next page
            driver.find_element_by_css_selector("a>.fa-angle-right").click()
        except:
            # If no more next page set the not end flag as True
            not_end = False
    return output

In [101]:
budgets = crawl("6: 'budget'")
df = pd.DataFrame.from_dict(budgets, orient='index')
df.to_csv('budgets.csv', index=False, encoding='utf-8-sig')

## Written Answers to Questions

In [110]:
def ans_qns(value):
    #Web Crawler Setup
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument("--start-maximized")
    driver = webdriver.Chrome(executable_path="Chrome/chromedriver.exe", options = chrome_options)

    # The website for the parliamentary info
    parliament_url = 'https://sprs.parl.gov.sg/search/home'

    #Access main Website
    driver.get(parliament_url)

    # Click on the filter tab
    driver.find_element_by_id("advSearchLabel").click()

    # Get the array of web elements with a form control
    forms = driver.find_elements_by_css_selector("select.form-control.ng-untouched")

    # Click on both the 13th and 14th to extract the data
    to_extract = Select(forms[0])

    # Select the dates that are important
    to_extract.select_by_value('13: 13')
    to_extract.select_by_value('14: 14')

    # Select the filter
    to_filter = Select(forms[4])

    # Select the section to be extracted
    to_filter.select_by_value(value)

    # Click on the search button
    driver.find_element_by_xpath("/html/body/app-root/app-search/div[@id='container']/div[@id='divmpscreen2']/div[@class='row']/div[@class='col-sm-12 text-right pull-right']/div[@class='col-sm-12 form-group']/button[@class='btn btn-black'][2]").click()

    # Wait for 3 seconds for the page to load
    driver.implicitly_wait(3)

    # Switch to the other tab with the information
    driver.switch_to.window(driver.window_handles[1])

    # Results dictionary
    results = {}

    # Flag to stop looping
    not_end = True

    count = 0

    # Loop until reach end of page
    while not_end:

        # Wait for 5 seconds
        driver.implicitly_wait(5)

        # Get all the webelements to be clicked later
        output = driver.find_elements_by_css_selector("tbody>tr>td>a")

        for links in output:
            links.click()
            
            count += 1
            
            print(count)
            
            # Switch to the content window
            driver.switch_to.window(driver.window_handles[2])

            # Delete those span with the procText class
            small_words = set([i.text for i in driver.find_elements_by_css_selector('span.procText')])

            # Get the title of the report
            title = driver.find_element_by_css_selector(".titleCaps").text.lower()

            # Get the content of the report
            content = driver.find_elements_by_css_selector('.reportTable > p')

            # Instantiate empty string as temp
            tmp = ''

            # Iterate through the content webelement to get the textual information
            for i in range(len(content)):
                if i == 0:
                    question = ' '.join(content[i].text.split()[1:])
                else:
                    if content[i].text not in small_words:
                        tmp += content[i].text

            # Fill the results
            results[count] = {'title': title, 'question': question, 'answer': tmp}

            # Close current page
            driver.close()

            # Go back to the main results page
            driver.switch_to.window(driver.window_handles[1])

        # Iterate through the pages
        try:
            # Try to go next page
            driver.find_element_by_css_selector("a>.fa-angle-right").click()
        except:
            # If no more next page set the not end flag as True
            not_end = False
        
    return results

In [111]:
written_qns = ans_qns("16: 'written-answer'")
written_qns

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


{1: {'title': 'initiatives to promote inter-generational collaboration and actions in tackling issues facing singaporeans',
  'question': 'Ms Ng Ling Ling asked the Minister for Culture, Community and Youth what are the existing platforms or initiatives that the Ministry has created to promote inter-generational collaboration and actions in tackling the issues facing Singaporeans, including the ageing population and shift in workforce composition going forward.',
  'answer': "Mr Edwin Tong Chun Fai: MCCY brings together Singaporeans from across generations to build consensus on issues that matter to them, and plan our future together. Some examples are as follows:(a) We engage Singaporeans to forge our shared identity, and to think about the values and legacy we want to leave behind for future generations. More than 80,000 people of all ages contributed their ideas and views to the Founders' Memorial to date.(b) This year, in spite of the on-going pandemic, 74 Singaporeans aged between

In [112]:
df = pd.DataFrame.from_dict(written_qns, orient='index')
df.to_csv('written_qns.csv', index=False, encoding='utf-8-sig')

In [113]:
written_qns_extended = ans_qns("17: 'written-answer-na'")
df = pd.DataFrame.from_dict(written_qns_extended, orient='index')
df.to_csv('written_qns_na.csv', index=False, encoding='utf-8-sig')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [114]:
df

,title,question,answer
1,criteria for setting up hawker centres and wet...,Mr Don Wee asked the Minister for Sustainabili...,"Ms Grace Fu Hai Yien: In 2011, the Government ..."
2,reasons for recent surge in dengue cases,Mr Liang Eng Hwa asked the Minister for Sustai...,21 Mr Yip Hon Weng asked the Minister for Sust...
3,statistics for covid-19 community testing,Mr Gerald Giam Yean Song asked the Minister fo...,15 Mr Gerald Giam Yean Song asked the Minister...
4,measures at autonomous universities to reduce ...,Ms Tin Pei Ling asked the Minister for Educati...,24 Dr Tan Wu Meng asked the Minister for Educa...
5,strategy for attracting foreign direct investm...,Mr Patrick Tay Teck Guan asked the Minister fo...,Mr Chan Chun Sing: Attracting foreign investme...
...,...,...,...
1668,review of arts fund guidelines,"Mr David Ong asked the Minister for Culture, C...",Ms Grace Fu Hai Yien: Administered by the Nati...
1669,update on hdb's goodwill repair assistance scheme,Mr Muhamad Faisal Bin Abdul Manap asked the Mi...,"Mr Lawrence Wong: Under the HDB lease, the rep..."
1670,monitoring lift maintenance and repair,Ms Sun Xueling asked the Minister for National...,Mr Lawrence Wong: Town Councils have the statu...
1671,supply and demand for pilots,Ms Tin Pei Ling asked the Minister for Transpo...,Mr Khaw Boon Wan: There are four approved pilo...
